# Домашнее задание №6

Асланов А.Б., ИУ9-21М

- Реализовать языковую модель информационного поиска


Факт 1: "В медленном движении медленно читают".    
Факт 2: "Петру I доставляли из Китая посылки с золотым песком".    
Факт 3: "Зоны отдыха экипажей дальнемагистральных авиарейсов находятся над или под пассажирским салоном".    

In [1]:
from pymystem3 import Mystem
m = Mystem()


def read_article(file):
    """Чтение статьи (на которую ссылается факт) из текстового файла"""
    
    with open(file, 'r') as f:
        fact = f.read()
        return fact


def make_morph_analysis(fact):
    """Проводим морфологический анализ для последующей обработки"""
    
    # Удлить все знаки препинания
    fact = ''.join(ch for ch in fact if ch not in "[«»;,()'`-—.?!:']")
    
    # Сделать все слова предложения в нижнем регистре
    fact = fact.lower()
    
    # Лемматизация
    fact = m.lemmatize(fact)
    fact = ''.join(fact)
    fact = fact.split('\n') # Разделение на предложения сохраняется
    fact = [sent for sent in fact if sent != ''] # Удаляем пустые предложения
    return fact


article1 = make_morph_analysis(read_article('fact1.txt'))
article2 = make_morph_analysis(read_article('fact2.txt'))
article3 = make_morph_analysis(read_article('fact3.txt'))
articles = [article1, article2, article3]
queries = ['В медленном движении медленно читают', 'Петру I доставляли из Китая посылки с золотым песком', 
           'Зоны отдыха экипажей дальнемагистральных авиарейсов находятся над или под пассажирским салоном']

queries = [make_morph_analysis(query) for query in queries]
print(queries)

[['в медленный движение медленно читать'], ['петр i доставлять из китай посылка с золотой песок'], ['зона отдых экипаж дальнемагистральный авиарейс находиться над или под пассажирский салон']]


# Языковые модели
       
**Идея: задаем в запросе слова, которые предполагаем увидеть в документе.**   
Хотим посмотреть, насколько документ $d_i$ мог бы породить запрос $q_i$.

По корпусу оцениваем возможность появления слов: на документах насчитываем вероятности слов (униграмм), а затем смотрим, какова вероятность запроса по отношению к документу.         
**Подход:** берем документ, берем каждое слово столько раз, сколько оно встречалось. Делим получившуюсь частотность на размер документа (количество слов). Теперь, когда у нас есть запрос, мы можем посчитать, какие у слов запроса были вероятности, и вывести вероятность того, что запрос соответствует нашему документу.

Общая формулировка языковой модели:
\begin{equation}
p(Q|d) = \prod\limits_{t \in Q}((1-\lambda)p(t) + \lambda p(t|M_d))
\end{equation}
Здесь первое слагаемое - модель коллекции, а второе - индивидуальная модель документа, $M_d$ - языковая модель документа $d$.    
Для данной задачи бралась $\lambda=0,5$.

In [2]:
import numpy as np
import pandas as pd
import collections
from collections import OrderedDict
c = collections.Counter()


article = articles[0]
# первый индекс выбирает факт (запрос), второй - преобразует список в строку.
query = queries[0][0].split() 
    
    
def count_words_frequency(sent, query):
    """Считаем количество вхождений слов из запроса в конкретное предложение"""
    
    c.clear() # обнуление счетчика подсчета слов
    sent = sent.split()
    for word in sent:
        c[word] += 1
        # если в предложении встречается слово, которого нет в запросе, то оно не должно добавляться в словарь
        if word not in query: 
            del c[word]
    for word in query:
        if word not in sent:
            c[word] = 0
    # возвращаем словарь, в котором слова (ключи словаря) стоят ранжированные по алфавиту
    return OrderedDict(sorted(c.items(), key=lambda t: t[0]))


def count_words_num(article):
    """Общее число слов"""
    
    N = 0
    for sent in article:
        N += len(sent.split())
    return N


def count_total_entries(article, query):
    """Считаем суммарное количество вхождений слов из запроса во все предложения"""
    
    all_words = []
    for sent in article:
        sent = sent.split()
        for word in sent:
            all_words.append(word)
    for word in all_words:
        if word in query:
            c[word] += 1
    return OrderedDict(sorted(c.items(), key=lambda t: t[0]))
        
    
# Число суммарных вхождений слов из запроса во все предложения статьи (на которую ссылается факт). 
total_entries = count_total_entries(article, query)
N = count_words_num(article)

sents = []
freqs = []
# Заполняем датафрейм: названиями колонок будут слова запроса
for sent in article:
    sents.append(sent)
    # freq = число вхождений слов в конкретное предложение статьи (на которую ссылается факт)
    freq = count_words_frequency(sent, query)
    freqs.append(freq)

# Добавляем в конец еще одну дополнительную строку с указанием общего числа встречаемости слова среди всех документов
freqs.append(total_entries)



df = pd.DataFrame(freqs, columns=query)
# Добавляем в конец еще одну дополнительную строку с указанием общего числа встречаемости слова среди всех документов
sents.append('ВСЕГО ВО ВСЕХ ДОКУМЕНТАХ СЛОВО ВСТРЕТИЛОСЬ (раз): ')
sents = pd.Series(sents)
df = df.set_index(sents)

In [3]:
from itertools import islice
from operator import add
LIMIT = len(df)-1 # учитываем все строки, кроме последней
lambda_ = 0.5

# Модцифицированный total_entries, 
# домноженный на (1-lambda) и разделенный на общее число документов в коллекции
total_entries = list(total_entries.values())
modified_total_entries = [x * (1-lambda_) / N for x in total_entries]

ps = []
# Пробегаем по строкам датафрейма
for idx, freqs in islice(df.iterrows(), LIMIT):
    freqs = freqs.tolist()
    # Делим каждую из частот на число термов конкретного предложения
    p = [lambda_ * x/len(idx.split()) for x in freqs]
    p = list(map(add, p, modified_total_entries))
    ps.append(np.product(p))
ps.append(None) # для последней строки мы не рассчитывали вероятности, оставим ячейку пустой для соразмерности столбцов.

df['p'] = ps
df = df.sort_values(by=['p'], ascending=False) # ранжируем вероятности по убыванию
df

,в,медленный,движение,медленно,читать,p
медленный чтение как часть медленный движение,0,2,1,0,0,7.812394e-09
в настоящий время существовать сайт медленный движение который предоставлять доступ к информация ресурс услуга и сетевой возможность все кто интересоваться изучение экологический мышление жизнь и взаимодействие в мировой сообщество создатель сайт утверждать что хороший способ быть часть медленный движение это читать хороший книга движение медленный книга англ slow books ставить свой цель возвращение к чтение книга доставлять истинный удовольствие,2,3,3,0,1,4.510637e-09
последователь методика медленный чтение не являться организовывать в свой движение нет ни один бланк ни совет директор ни о ужас центральный сайт как писать американский журналист малкольм джонс в статья для newsweek однако теоретик метод утверждать что идея медленный чтение зарождаться гораздо рано появление медленный движение,2,3,2,0,0,1.084396e-09
медленный чтение этот намеренный снижение скорость чтение цель который улучшение понимание текст или получение удовольствие метод медленный чтение позволять более полно понимать и оценивать сложный текст при изучение философия и литература однако в последний время наблюдаться повышенный интерес к медленный чтение как направление в рамка медленный движение,2,4,1,0,0,6.987584e-10
профессор хьюстонский университет дэвид микикс в книга медленный чтение в торопливый эпоха англ slow reading in a hurried age описывать история медленный чтение утверждать что идея существовать примерно с 200 год н э когда раввин читать и обсуждать библейский текст однако современный принцип движение получать свой развитие около шестьдесят год назад в гарвардский университет где рубен бауэр англ reuben brower и коллега обучать студент техника вдумчивый чтение и анализ выбор автор определенный слово в контекст произведение чтение по их мнение требовать достаточно время чтобы познакомиться с книга понимать она ритм и атмосфера,4,2,1,0,1,4.512870e-10
трейси сили англ tracy seeley преподаватель английский язык из университет санфранциско и автор блог о медленный чтение считать что медленный чтение не должно быть занятие только для интеллектуал хотя этот движение и возникать в академический круг тщательный медленный чтение и глубокий внимание это вызов для весь мы,1,3,1,0,0,3.822432e-10
карл оноре сторонник медленный движение в книга без суета как переставать спешить и начинать жить англ in praise of slow how a worldwide movement is changing the cult of speed рекомендовать медленный чтение как один из практика отвлечение от быстрый темп современный жизнь,1,2,1,0,0,3.259715e-10
канадский автор джон мьедема англ john miedema в книга медленный чтение англ slow reading ссылаться на работа карла оноре и сравнивать медленный чтение с медленный еда сторонник движение медленный еда предпочитать местный продукт и кулинарный традиция аналогично концепция медленный чтение мочь быть расширять за счет продвижение публичный учреждение культура произведение местный автор он подчеркивать что медленный чтение это не постоянный чтение в максимально медленный темп но скоро право на сознательный замедление медленный чтение по мнение мьедема это философия понимание и наслаждение и набор инструмент для глубокий работа с произведение для соединение идея автор с идея читатель и индивидуальный переживание книга а не последствие диагностировать заболевание такой как дислексий или нарушение зрение,2,8,1,0,0,1.895654e-10
техника медленный чтение,0,1,0,0,0,9.520859e-11
в книга медленный чтение джон мьедема писать что медленный чтение должно быть персонализировать то быть читатель рекомендоваться адаптировать метод к свой индивидуальный стиль потребность и обстоятельство что подразумевать в частность замедление только в определять место текст субвокализация спор с текст и актерский воспроизведение некоторый фрагмент,3,2,0,0,0,4.590320e-11


In [4]:
for i in range(len(queries)):
    with open(f'{queries[i][0]}.txt', 'w') as f:
        f.write(str(df['p'][:-1])) # не включаем последнюю строчку, потому что в ней нет вероятностей

# Вывод

Для факта **В медленном чтении медленно читают** наиболее подходящие результаты выдачи, согласно языковой модели, следующие:

1) **7.812394e-09** медленный чтение как часть медленный движение   


2) **4.510637e-09** последователь методика медленный чтение не являться организовывать в свой движение нет ни один бланк ни совет директор ни о ужас центральный сайт как писать американский журналист малкольм джонс в статья для newsweek однако теоретик метод утверждать что идея медленный чтение зарождаться гораздо рано появление медленный движение                                                               

3) **1.084396e-09** медленный чтение  этот намеренный снижение скорость чтение цель который  улучшение понимание текст или получение удовольствие метод медленный чтение позволять более полно понимать и оценивать сложный текст при изучение философия и литература однако в последний время наблюдаться повышенный интерес к медленный чтение как направление в рамка медленный движение 

------------------------------------------------------------------------------------------------------

Для факта **Петру I доставляли из Китая посылки с золотым песком** наиболее подходящие результаты выдачи:    

1) **1.965507e-10** коробчатый золото  золотой песок который поставляться в российский империя из китай для чеканка червонец золотой песок упаковывать в небольшой коробочка отчего и происходить его название коробчатый золото использоваться для чеканка червонец в 1701 и 1704 год      

2) **1.051329e-10** золотой песок упаковывать в коробочка небольшой размер в каждый из такой коробочка находиться по 3/4 фунт золотой песок из китай поступать в основной золотой песок а золотой слиток привозить в намного меньший количество в 1701 год коробчатый золото использовать для чеканка 9735 червонец на чеканка этот деньги из казна потратить 11141 рубль деньги пойти на оплата работа мастер стоимость самый золото потеря при плавка каждый червонец в среднее обходиться казна в 1 рубль 14 копейка в 1704 год коробчатый золото вновь использоваться для чеканка русский червонец который быть все отчеканивать 8675 расход на каждый из они составлять 1 рубль 30 копейка вес червонец 1701 год составлять 347 г из который 336 г  чистый золото диаметр  2324 мм двойной червонец весить 694 г из который чистый золото  672 г диаметр  24 мм    

3) **3.644107e-11** по один данные коробчатый золото поступать на монетный двор через сибирский приказ использоваться иногда для чеканка двухрублевик а по другой сведение в февраль 1718 год золотой двухрублевик становиться чеканить из лигатурный золото  стопа и проба монета быть изменять золотой двухрублевик который также называть андреевский золотой изза размещение на один из сторона изображение апостол андрей первозванний становиться чеканиться из лигатурный золото видимо изза рост цена на коробчатый золото 


------------------------------------------------------------------------------------------------

Для факта **Зоны отдыха экипажей дальнемагистральных авиарейсов находятся над или под пассажирским салоном**  наиболее подходящие результаты выдачи:

1) **5.838329e-21** отсек для отдых экипаж обычно присутствовать на дальнемагистральный лайнер и мочь располагаться над пассажирский салон в этот случай в они нужно подниматься по лестница или рядом с он секция для пилот располагать в передняя часть самолет отделять от располагать сзади секция для бортпроводник                                                                                                 

2) **3.849288e-21** зона отдых экипаж  отсек или специально отводить место внутри самолет предназначать для отдых член экипаж при выполнение дальний рейс поскольку максимально допустимый продолжительность работа экипаж ограничивать авиационный регламент а иногда также трудовой законодательство рейс по длительность превышать этот время выполняться со сменный экипаж член экипаж мочь воспользоваться зона отдых в нерабочий время                                                                                                                     

3) **3.207447e-22** faa определять 3 класс зона отдых который должный предоставляться в зависимость от количество член экипаж и продолжительность рейс           

------------------------------------------------------------------------------------------------